# Improved LSTM baseline

This kernel is a somewhat improved version of [Keras - Bidirectional LSTM baseline](https://www.kaggle.com/CVxTz/keras-bidirectional-lstm-baseline-lb-0-051) along with some additional documentation of the steps. (NB: this notebook has been re-run on the new test set.)

In [ ]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

We include the GloVe word vectors in our input files. To include these in your kernel, simple click 'input files' at the top of the notebook, and search 'glove' in the 'datasets' section.

In [ ]:
path = '../input/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE=f'{path}glove6b50d/glove.6B.50d.txt'
TRAIN_DATA_FILE=f'{path}{comp}train.csv'
TEST_DATA_FILE=f'{path}{comp}test.csv'

In [ ]:
TEST_MODI = "minister for agriculture under the Member of Parliament Professor E. He and honorable MLA shri VP Sajendaran. It was from here that the great indian sage embark the journey across india to protect indian civilization.It is indeed a proud moment not just for kerela but fom nation.Bharat refinary has played a critical role in refining petrol for kerela and india for 50 years.Petroleum Corporation William Corporation limited under the ministry of petroleum and natural gas. It's my privilege to expand everyone about integrated Refinery expansion complex. I could see Russia and the mountain storage facility of Indian Oil Corporation. I couldn't bottling plant will also witness the Foundation Stone Mountain Complex irep of being constructed at an approximate cost of 16504 crore rupees. With world-class facilities the dedication also marks and my stone of making eco-friendly Board of State for Fears accessible to the whole of South India by the presence of an initiate for the development great the song of Kayla Rodriguez. becoming prime minister alcohol respect and gratitude for gracing this occasion. What does towering presence? Let's also present d.com are the oil companies aims at providing vocational training to the youth and enhance the employability and Entrepreneurship in oil and gas industry address. The esteem. Yes, and audien.Going to justice. Peace if possible. to finish time April Drive Eng Michael Egan Union doorman cell phones MP3 JB Thomas MVP episode attorney MLS Libby Pizza in Durand Cordelia sahadi's hours Helena, Montana I'm happy to be here at Kochi. the game of Arabian Sea the blue sea the back marker the Great Barrier Reef The Greenery all around how does dynamic people make protein.a queen among the city, It was from here. that the greatest Indian Sage an Epic Journey Across, India  to protect the Indian civilization today is a historic day. when the largest industrial unit of Kerala is entering in next phase of development. It is indeed. a proud moment Not just for Kerala but also for entire nation . Bharat petroleum kochin refinary has played a critical role in popularizing the clean fuel LPG Among Masses in Kerala and neighboring states for more than 50 years. I recall my childhood. when I had seen Many mother struggling with the firewood stove in the kitchen ever since i've always though of  improving their situation and providing healthy kitchen to mother and sisters of India. Ujwalla scheme of government of india is a way to realize the dream i'm happy that since may 2016 nearly six crores LPG connections have been provided to the poorest of the poor in the country. friends More than 23 crores LPG consumers has joined the pehel scheme , pehel has helped in identifying ghost accounts, multiple accounts and inactive accounts. pehel has entered  into the Guinness Book of World Records at the largest direct benefit transfer scheme in the world. over one crore customer have given up LPG subsidy under giveitup initiative. by doubling the LPG production with the help of recent expansion kochi refinary is making great contribution to ujjwala. to over come environment pollution, The government of India is promoting the use of Land Management friendly transport. That is CNG by expanding the coverage of city gas Distribution Network in the country. After the successful completion of 10 cjd bidding round, More than 400 distance of country. will be connected for providing pipe gas supply. the national greed or pradhan mantri urja ganga  Has also been created to have gas basic economy and enhance the share of gas in the energy basket. The government has thought of developing additional 50000 km of gas pipeline network to cut down on import of crude oil. Government has taken decisive steps towards reducing import by 10% and saving precious foreign exchange, establishing 12 2-g ethanol plants. The Indian refined industry at done well in establishing as a major player globally. India which is the second largest oil refinery in Asia is emerging as  Refinery hub with refining more than its demand the country  capacity is currently more 247 mmtpa. I think this opportunity. to congratulate all part of the timely completion of IREP. Last but not least. I recognize. the work of all those labourers who work day and night during construction. I'm told that at the peak of the project over 20k workers working at the site here are the real heroes of project. integrated Refinery Expansion Project was Also  strategic move of bharat petroleum to diversify into non fuel sector . petrochemicals are a grade of chemicals which we not speak much about But they exist invisibly and touch many aspects of a daily life this includes building materials plastic and paints , clothings, Clothing and other fabrics, automatic parts cosmetic and medicines. most of these chemicals are imported from other countries It is our Endeavour to see that these petrochemicals are manufactured in India itself. I'm happy that using the capability of refinary to produce propylene after execution of IRIP. BPCL had gone ahead with putting up 3 world scale plant to manufacture acrylic and also ethanol under make in India. petrochemicals are used in paint coating. bpcl is starting the construction of a petrochemical complex where it would produce polyoil that find use in cosmetic and Medicine. I'm sure by all these many  industry would come to Kochi. I hope the petrochemical park plan by state government would become operational soon and Liberate the business opportunities provided by the petrochemical Ventur of Bpcl. I'm happy to know that bpcl along with other PSU and developing employee youth. I'm delighted to lay the foundation stone of the second Campus of the institute at atmanor near the Holy Temple I'm also glad to know that Indian Oil Corporation storage facility around 10 km from here at cost of 50 crores. This will enhance LPG storage capacity and also reduce road moevement of lpg tankers, last August when Kerala was passing through the worst flood in a hundred years. bpcl Kochi refinary was running Against All Odds I understand. Then many employees stayed in the refinary to ensure continuous production. bpcl Kochi business binder to keep up the spirit of Harbour social commitment and Innovation about its next stage of development. We are proud of cochina refinary contribution to our nation building. But now we have an explanation, I wish that kochi refinary leads petrochemical revolution in Southern India and support the growing needs of a new India. Thank you. Thank you very much.A great retail with breaking ties and listen, ladies and gentlemen, we come to the end of the formal functions on behalf of all of this present a heartfelt gratitude for attending dysfunction and making a truly memorable with your graceful presents and eloquent speech a big. Thank you from the bottom of heart."

In [ ]:
TEST_UNITED = "in coming days modi babu sarkar has come to expiry date. coming days of modi sarkar has came to end. younger generation has very strength. bhartiya janta party you are decoits. if speaking truth is a revolt then i am a revolt . if 136 aircrafts were neccesary why only 36. why not given to hal for aircraft contact. watchman is theif theif watchman is theif. bangal man is intelligent . bangal man is intellectual. you said 56 inch chest. show your hear. tell people demonitisation is wrong action. country was not ready for gst. farmers are getting looted. hearty congratualtion to unite and creation tension in modi sleep. modi is not gonna apologize. fight against bjp and throw him out of the country. watchman should know people of country is lawyer of your action. today bjp neta use abusive words for womens . modi is following wrong people on twitter . mob lynching everywhere in poeple, muslims are in great threat . modi has break the country which has not happened in 70 years from independence . doing peices of this country modi and amit . jammu kashmir responsible is also bjp for death of people and threat . i am muslim but i am part of this country. we want to live with the poeple of this country and die with the people of this country. we are all aware 5 years back people gave mandate to bjp and nda . they betrayed the nation they gave all these slogans mudra, jandhan , corrupt free country. we want performing prime minister. those people say they have some many poeple for pm candidate . they try to throw sarcasm on us that they have many grooms. we ask you , those who have failed those who betrayed, spread hatred in society those who pranksters who else other than bjp. they have saddened the country. farmer is only verge on suicide. this situation is developed who whom . demonitisation is taken country a decade in the history. businessman has gone bankrupt."

In [ ]:
from nltk import sent_tokenize

In [ ]:
modi_speech = sent_tokenize(TEST_MODI)
rally_speech = sent_tokenize(TEST_UNITED)

In [ ]:
len(rally_speech)

Set some basic config parameters:

In [ ]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

Read in our data and replace missing values:

In [ ]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

Standard keras preprocessing, to turn each comment into a list of word indexes of equal length (with truncation or padding as needed).

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
modi_train = tokenizer.texts_to_sequences(modi_speech)
rally_test = tokenizer.texts_to_sequences(rally_speech)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
XX_t = pad_sequences(modi_train, maxlen=maxlen)
XX_te = pad_sequences(rally_test, maxlen=maxlen)

Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

In [ ]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

Use these vectors to create our embedding matrix, with random initialization for words that aren't in GloVe. We'll use the same mean and stdev of embeddings the GloVe has when generating the random init.

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

Simple bidirectional LSTM with two fully connected layers. We add some dropout to the LSTM since even 2 epochs is enough to overfit.

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Now we're ready to fit out model! Use `validation_split` when not submitting.

In [ ]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1);

And finally, get predictions for the test set and prepare a submission CSV:

In [ ]:
modi_prediction = model.predict([XX_t])

In [ ]:
united_rally_prediction = model.predict([XX_te])

**Now, Save these two objects in csv and calculate mean of all sentences to summary the output for each class.**